In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd
import numpy as np
import csv
import FL_utils.FL_baseline_functions as base

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined_test\FL Model


In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/KY_cleaned").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined_test/data preparation/FL_cleaned").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['general_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['general_two_year'].values

### Model

In [8]:
#### Logistic
c = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 3e-5, 1e-2, 5e-2]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### Lasso
c = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 3e-5, 1e-2, 5e-2]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### LinearSVM
c = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 3e-5, 1e-2, 5e-2]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### Random Forest
n_estimators =  [50,100,150,200,250]
depth = [1,2,3]
impurity = [0.001, 0.002, 0.003, 0.004, 0.005, \
            0.006, 0.007, 0.008, 0.009, 0.01]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y,  
                     depth=depth, 
                     estimators=n_estimators, 
                     impurity=impurity,
                     seed=816)

#### XGBoost
learning_rate = [0.05]
depth = [1,2,3]
n_estimators = [40,60,80,100,150,200,250]
gamma = [6,8,10,12]
child_weight = [6,8,10,12]
subsample = [0.5]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y, 
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       gamma=gamma, 
                       child_weight=child_weight, 
                       subsample=subsample, 
                       seed=816)

#### save results
summary_general2_fl_model = {"Logistic": logistic_summary,
                             "Lasso": lasso_summary,
                             "LinearSVM": svm_summary,
                             "RF": rf_summary,
                             "XGBoost": xgb_summary}

In [15]:
results = []
for model_name, model_summary in summary_general2_fl_model.items():
    results.append([model_name, 
                    model_summary['best_auc'], 
                    model_summary['auc_diff'], 
                    model_summary['KY_score']])
results

[['Logistic', 0.6665017953245881, 0.023239727365785723, 0.6491904740708001],
 ['Lasso', 0.6656391702868748, 0.017856582343749317, 0.6409083719472976],
 ['LinearSVM', 0.6665597863358662, 0.018587581117770147, 0.6388943105753853],
 ['RF', 0.6506237291004032, 0.02790727110715241, 0.6718355105250913],
 ['XGBoost', 0.6642601837408469, 0.0231679123447871, 0.6608851043028511]]

In [17]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\baselines\\Two Year\\"
results = [["", "Logistic", "Lasso", "Linear SVM", "Random Forest", "XGBoost"],
           ["General",             
            round(logistic_summary['KY_score'],3), 
            round(lasso_summary['KY_score'], 3), 
            round(svm_summary['KY_score'],3), 
            round(rf_summary['KY_score'],3), 
            round(xgb_summary['KY_score'],3)]]
with open(path + 'Two Year FL_model.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)